In [ ]:
from tiled_multiprocess import run_main


## Select file(s)

List paths to files. Paths can be passed as plain string format or as a `pathlib.Path` object.

In [ ]:
# List of INPUT DATASETS (have to be in this format)
list_tifs = [
    r"<PATH_TO_INPUT_FILE>"
]

**save_float**
* The defaults are saved in 8-bit format (256 values, RGB)
* To also save pixel values as float, change the `save_float` parameter to `True`

**save_vrt** 
* if `True` the results are left as tiles and a VRT file is created
* if `False` results are merged into TIF mosaic and indvidual tiles are deleted

In [ ]:
save_float = True
save_vrt = True

## Select visualizations

Select the desired blends by listing the keywords. Leave an empty Python list if no visualizations are to be saved.

```
vis_types = ["slrm", "hs"]
```
example of empty list:
```
vis_types = []
```

Keywords for basic visualizations:
*  'slp' - Slope
*  'hs' - Hillshade
*  'slrm' - Simple local relief model
*  'svf' - Sky view factor
*  'opns' - Positive openness
*  'neg_opns' - Negative openness
*  'ld' - Local dominance
*  'mstp' - Multi-scale topographic position

> All visualizations are saved as 8-bit images using the default values for stretch and cut off

In [ ]:
# Select visualizations
vis_types = [
    
]

## Select blends

Select the desired blends by listing the keywords:

```
blend_types = ["vat_combined"]
```
example of empty list:
```
blend_types = []
```

Choose from the following keywords for blends:
*  'vat_combined'
*  'e2MSTP'
*  'e4MSTP'
*  'CRIM'

> All blends are saved as 8-bit images using the default values for stretch and cut off

In [ ]:
# Select visualizations
blend_types = [
    "vat_combined"
]

## Run the main routine

Start processing by executing the following cell:

In [ ]:
run_main(
    list_tifs=list_tifs,
    vis_types=vis_types,
    blend_types=blend_types,
    save_float=save_float,
    save_vrt=save_vrt
)